In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
import matplotlib.dates as dts
import numpy as np
import pandas as pd
import itertools
import os
import ROOT
import datetime
from root_numpy import root2array, root2rec, tree2rec, array2root
from scipy.optimize import curve_fit
from scipy.misc import factorial
plt.rcParams.update({'font.size': 16})
#from larlite import larlite
#k=larlite.trigger()
from ROOT import larlite
larlite.trigger()
from ROOT import trigger
from fememu_pycfg import apply_config


In [ ]:
# emulator configuration
cfg_file = 'debug_teststand.cfg'
config = trigger.fememu.FEMBeamTriggerConfig()
apply_config(config,cfg_file)
# emulator construction
emu = trigger.fememu.LLInterface(config)
# data file loading
f_trig = ROOT.TFile('debug_trig.root')
f_xmit = ROOT.TFile('debug_fifo.root')
t_trig = f_trig.Get('trigger_daq_tree')
t_xmit = f_xmit.Get('fifo_pmt_xmit_tree')
print 'trigger entries :',t_trig.GetEntries()
print 'xmit entries    :',t_xmit.GetEntries()


In [ ]:
trig_times = {}
for x in xrange(t_trig.GetEntries()):
    t_trig.GetEntry(x)
    trig = t_trig.trigger_daq_branch
    trig_num  = trig.TriggerNumber()
    trig_time = trig.TriggerTime()
    trig_frame = int(trig_time /1600.)
    trig_sample = (trig_time - trig_frame * 1600.) / (1./64.)
    trig_times[trig.TriggerNumber()] = (trig_time,trig_frame,trig_sample)

# find first saturation tick number
def getFireTick(vec):
    for idx in xrange(len(vec)):
        if idx < 3: continue
        if vec[idx] - vec[idx-3] > 100: 
            return idx

In [ ]:
show_ch1 = False
fig = plt.figure(figsize=(15,6))
# loop through xmit info
for x in xrange(t_xmit.GetEntries()):
    t_xmit.GetEntry(x)
    t_trig.GetEntry(x)
    ev_fifo = t_xmit.fifo_pmt_xmit_branch
    frame_num = sample_num = 0
    
    ch0,ch1,beamch=(None,None,None)

    for n in xrange(ev_fifo.size()):
        fifo = ev_fifo[n]
        if not fifo.module_address() == 5: continue
        channel = fifo.channel_number()
        
        if channel == 0: 
            ch0 = fifo
        elif channel == 1:
            ch1 = fifo
        elif channel in [46,47]:
            beamch = fifo

    ch0_adc = np.array(ch0)
    ch1_adc = np.array(ch1)
    beamch_adc = np.array(beamch)

    # Get timings
    beamgate_sample = ch0.readout_sample_number_RAW()
    
    trig_num = ev_fifo.event_number()
    trig_time,trig_frame,trig_sample = trig_times[trig_num]
    trig_tick = trig_sample - beamgate_sample

    beamch_tick = beamch.readout_sample_number_RAW() - beamgate_sample
    beamch_fire = getFireTick(beamch_adc)

    if trig_tick < 0 or trig_tick > 100: continue
        
    plt.plot(ch0,label='Ch. 0 ADC Values',color='b')
    
    plt.axvline(trig_tick,color='g',lw=2,label='Trigger (Tick %d)' % trig_tick)
            
    # get saturation tick for ch1
    ch0_fire = getFireTick(ch0_adc)
    plt.plot(ch0_fire,ch0_adc[ch0_fire],'go',label='Ch. 0 PHMAX > 100 @ %d' % ch0_fire)
    if show_ch1:
        ch1_fire = getFireTick(ch1_adc)
        plt.plot(ch1,label='Ch. 1 ADC Values ',color='r')
        plt.plot(ch1_fire,ch1_adc[ch1_fire],'go',label='Ch. 1 PHMAX > 100 @ %d' % ch1_fire)

    for x in xrange(100):
        plt.axvline(x*5,color='k',alpha=0.5,linestyle='--')
        
    #print ch0_fire,trig_tick
    # Emulator
    emu_out = emu.Emulate(ev_fifo)
    print 'Emulator fire @',emu_out.fire_time
    emu_fired = emu_out.fire_time >= 0
    
plt.grid()
#plt.legend()
plt.xlim(50,100)
plt.xlabel('Tick Number [64 MHz]')
plt.ylabel('ADC Counts')
plt.title('Waveforms')
plt.show()


In [ ]:
show_ch1 = False

# loop through xmit info
for x in xrange(t_xmit.GetEntries()):
    t_xmit.GetEntry(x)
    t_trig.GetEntry(x)
    ev_fifo = t_xmit.fifo_pmt_xmit_branch
    frame_num = sample_num = 0
    
    ch0,ch1,beamch=(None,None,None)

    for n in xrange(ev_fifo.size()):
        fifo = ev_fifo[n]
        if not fifo.module_address() == 5: continue
        channel = fifo.channel_number()
        
        if channel == 0: 
            ch0 = fifo
        elif channel == 1:
            ch1 = fifo
        elif channel in [46,47]:
            beamch = fifo

    ch0_adc = np.array(ch0)
    ch1_adc = np.array(ch1)
    beamch_adc = np.array(beamch)

    # Get timings
    beamgate_sample = ch0.readout_sample_number_RAW()
    
    trig_num = ev_fifo.event_number()
    trig_time,trig_frame,trig_sample = trig_times[trig_num]
    trig_tick = trig_sample - beamgate_sample

    beamch_tick = beamch.readout_sample_number_RAW() - beamgate_sample
    beamch_fire = getFireTick(beamch_adc)

    if trig_tick < 0 or trig_tick > 100: continue
        
    # Emulator
    emu_out = emu.Emulate(ev_fifo)
    print 'Emulator fire @',emu_out.fire_time
    emu_fired = emu_out.fire_time >= 0

    if emu_fired: continue
        
    #
    # Data plot
    #
    fig = plt.figure(figsize=(15,6))
        
    plt.plot(ch0,label='Ch. 0 ADC Values',color='b')
    
    plt.axvline(trig_tick,color='g',lw=2,label='Trigger (Tick %d)' % trig_tick)
            
    # get saturation tick for ch1
    ch0_fire = getFireTick(ch0_adc)
    plt.plot(ch0_fire,ch0_adc[ch0_fire],'go',label='Ch. 0 PHMAX > 100 @ %d' % ch0_fire)
    if show_ch1:
        ch1_fire = getFireTick(ch1_adc)
        plt.plot(ch1,label='Ch. 1 ADC Values ',color='r')
        plt.plot(ch1_fire,ch1_adc[ch1_fire],'go',label='Ch. 1 PHMAX > 100 @ %d' % ch1_fire)

    for x in xrange(100):
        plt.axvline(x*5,color='k',alpha=0.5,linestyle='--')
   
    plt.grid()
    plt.legend()
    plt.xlim(50,100)
    plt.xlabel('Tick Number [64 MHz]')
    plt.ylabel('ADC Counts')
    plt.title('Waveforms')
    plt.show()
    
    #
    # Emulator plot
    #
    fig = plt.figure(figsize=(15,6))

    phmax = emu.PHMAX()
    diff3 = emu.PhaseDiffDisc3(0)
    
    plt.plot(phmax,color='g',label='PHMAX')
    plt.plot(diff3,color='b',label='Ch. 0 Phase Diff.')
    for x in xrange(100):
        plt.axvline(x*5,color='k',alpha=0.5,linestyle='--')
    
    plt.grid()
    plt.legend()
    plt.xlim(50,100)
    plt.xlabel('Tick Number [64 MHz]')
    plt.ylabel('ADC Counts')
    plt.title('Waveforms')
    plt.show()


In [ ]:
show_ch1 = False
fig = plt.figure(figsize=(15,6))
# loop through xmit info
for x in xrange(t_xmit.GetEntries()):
    t_xmit.GetEntry(x)
    t_trig.GetEntry(x)
    ev_fifo = t_xmit.fifo_pmt_xmit_branch
    frame_num = sample_num = 0
    
    ch0,ch1,beamch=(None,None,None)

    for n in xrange(ev_fifo.size()):
        fifo = ev_fifo[n]
        if not fifo.module_address() == 5: continue
        channel = fifo.channel_number()
        
        if channel == 0: 
            ch0 = fifo
        elif channel == 1:
            ch1 = fifo
        elif channel in [46,47]:
            beamch = fifo

    ch0_adc = np.array(ch0)
    ch1_adc = np.array(ch1)
    beamch_adc = np.array(beamch)

    # Get timings
    beamgate_sample = ch0.readout_sample_number_RAW()
    
    trig_num = ev_fifo.event_number()
    trig_time,trig_frame,trig_sample = trig_times[trig_num]
    trig_tick = trig_sample - beamgate_sample

    beamch_tick = beamch.readout_sample_number_RAW() - beamgate_sample
    beamch_fire = getFireTick(beamch_adc)

    if trig_tick < 100 and trig_tick>0: continue
        
    plt.plot(ch0,label='Ch. 0 ADC Values',color='b')
    #plt.plot(ch1,label='ch1',color='g')
    
    #plt.axvline(trig_tick,color='r',lw=2,label='Trigger (Tick %d)' % trig_tick)
            
    # get saturation tick for ch1
    ch0_fire = getFireTick(ch0_adc)
    plt.plot(ch0_fire,ch0_adc[ch0_fire],'go',label='PHMAX > 100 @ %d' % ch0_fire)
    
    if show_ch1:
        ch1_fire = getFireTick(ch1_adc)
        plt.plot(ch1,label='Ch. 1 ADC Values ',color='r')
        plt.plot(ch1_fire,ch1_adc[ch1_fire],'go',label='Ch. 1 PHMAX > 100 @ %d' % ch1_fire)
    
    for x in xrange(100):
        plt.axvline(x*5,color='k',alpha=0.5,linestyle='--')
            
            
    # Emulator
    emu_out = emu.Emulate(ev_fifo)
    print 'Emulator fire @',
    for t in emu_out.fire_time_v:
        print t,
    print
    
plt.grid()
#plt.legend()
plt.xlim(50,100)
plt.xlabel('Tick Number [64 MHz]')
plt.ylabel('ADC Counts')
plt.title('Waveforms')
plt.show()

In [ ]:
# loop through xmit info
fire_time_v = []
show_ch1 = False
for x in xrange(t_xmit.GetEntries()):
    t_xmit.GetEntry(x)
    t_trig.GetEntry(x)
    ev_fifo = t_xmit.fifo_pmt_xmit_branch
    frame_num = sample_num = 0
    
    ch0,ch1,beamch=(None,None,None)

    for n in xrange(ev_fifo.size()):
        fifo = ev_fifo[n]
        if not fifo.module_address() == 5: continue
        channel = fifo.channel_number()
        
        if channel == 0: 
            ch0 = fifo
        elif channel == 1:
            ch1 = fifo
        elif channel in [46,47]:
            beamch = fifo

    ch0_adc = np.array(ch0)
    ch1_adc = np.array(ch1)
    beamch_adc = np.array(beamch)

    #plt.plot(ch1,label='ch1',color='g')

    # Get timings
    beamgate_sample = ch0.readout_sample_number_RAW()
    
    trig_num = ev_fifo.event_number()
    trig_time,trig_frame,trig_sample = trig_times[trig_num]
    trig_tick = trig_sample - beamgate_sample

    beamch_tick = beamch.readout_sample_number_RAW() - beamgate_sample
    beamch_fire = getFireTick(beamch_adc)
            
    # get saturation tick for ch1
    ch0_fire = getFireTick(ch0_adc)
    if ch0_fire in fire_time_v: continue
        
    fig = plt.figure(figsize=(15,6))
    plt.axvline(trig_tick,color='r',lw=2,label='Trigger (Tick %d)' % trig_tick) 

    plt.plot(ch0,label='Ch. 0 ADC Values',color='b')
    fire_time_v.append(ch0_fire)
    plt.plot(ch0_fire,ch0_adc[ch0_fire],'go',label='Ch. 0 Disc3 @ %d' % ch0_fire)

    if show_ch1:
        ch1_fire = getFireTick(ch1_adc)
        plt.plot(ch1,label='Ch. 1 ADC Values ',color='r')
        plt.plot(ch1_fire,ch1_adc[ch1_fire],'go',label='Ch. 1 Disc3 @ %d' % ch1_fire)
    
    for x in xrange(100):
        plt.axvline(x*5,color='k',alpha=0.5,linestyle='--')

    print 'BeamCh Start     @', beamch_tick
    print 'BeamCh PHMax>100 @', beamch_tick + beamch_fire
    print 'Ch. 0 fire       @', ch0_fire
    print 'Trigger fire     @', trig_tick
            
    plt.grid()
    plt.legend()
    plt.xlim(50,100)
    plt.xlabel('Tick Number [64 MHz]',fontsize=20)
    plt.ylabel('ADC Counts',fontsize=20)
    plt.title('Waveforms')
    plt.show()